# Rename the SAX/AX 3D dicom files according to the phase info

In [2]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
Console_and_file_logger('merge_data/rename_dicom_files', logging.INFO)

import SimpleITK as sitk
import random
from collections import Counter
from ipywidgets import interact

%matplotlib inline
plt.rcParams.update({'font.size': 30})
%reload_ext autoreload
%autoreload 2
pd.options.display.max_columns = None

from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.visualization.Visualize import plot_3d_vol, plot_4d_vol, plot_value_histogram, show_2D_or_3D
from src.data.Dataset import get_metadata_maybe, filter_4d_vol, copy_meta_and_save, create_3d_volumes_from_4d_files, describe_sitk, describe_volume, describe_path, get_phase, is_patient_in_df, get_extremas


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


2020-06-12 08:53:45,585 INFO -------------------- Start --------------------
2020-06-12 08:53:45,586 INFO Working directory: /mnt/data/git/cardio.
2020-06-12 08:53:45,586 INFO Log file: ./logs/merge_data/rename_dicom_files.log
2020-06-12 08:53:45,587 INFO Log level for console: INFO
Using TensorFlow backend.


In [3]:
# Interativ dataframe chooser
# select the path to the dicom dataframe
from ipyfilechooser import FileChooser
df_chooser = FileChooser(os.path.join(os.getcwd(),'data/raw/gcn_05_2020_ax_sax_86'), '')
display(df_chooser)

FileChooser(path='/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86', filename='', show_hidden='False')

In [28]:
# read 4D dataframe
df = pd.read_csv(df_chooser.selected)
print(df.shape)
df.head()

(429, 14)


,patient,x_path,y_path,t,YYYY,MM,DD,ID,ED#,MS#,ES#,PF#,MD#,phase
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,ES
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,15,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,PF
2,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,21,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,MD
3,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,ED
4,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,5,23,0HQQW4ZN,25.0,7.0,11.0,16.0,22.0,MS


In [29]:
# helper to replace the t0-9 substring in the x and y file names with the phase
def get_new_name(full_filename, phase):
    p, f = os.path.split(full_filename)
    f_ = re.sub('__t[0-9]+', '_{}'.format(phase), f)
    return os.path.join(p, f_)

def rename_x_path_y_path(row):
    import re
    phase = row['phase']
    
    old_x = row['x_path']
    old_y = row['y_path']
    new_x = get_new_name(old_x, phase)
    new_y = get_new_name(old_y, phase)
    try:
        os.rename(old_x, new_x)
        os.rename(old_y, new_y)
    except Exception as e:
        logging.debug(str(e))
    return new_x, new_y
    
    

In [30]:
# rename the x and y path of the dicom files and save the new file names to the dataframe
df['x_path'], df['y_path'] = zip(*df.apply(rename_x_path_y_path, axis=1))

In [31]:
df.to_csv(df_chooser.selected, index=False)